In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import os
import time
import pathlib

In [4]:
python_path = pathlib.Path('.').absolute().parent/'python'
os.sys.path.insert(1, str(python_path))

In [5]:
import pinocchio as pin
from robot_properties_kuka.config import IiwaConfig

import meshcat
import meshcat.transformations as tf
import meshcat.geometry as g

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from tqdm import trange
import wandb

In [6]:
from vocam.qpnet import DataUtils, QPNet
import vocam.qpnet

In [7]:
robot = IiwaConfig.buildRobotWrapper()
model, data = robot.model, robot.data
f_id = model.getFrameId("EE")

viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [9]:
run = wandb.init(project="vocam", group="kuka_qpnet_train", entity="hjzhu")

wandb.config.nq = model.nq
wandb.config.nv = model.nv
wandb.config.q0 = [np.pi/16.0, -np.pi/16.0, 0, 0, 0, 0, 0]
wandb.config.x_init = np.concatenate([wandb.config.q0, pin.utils.zero(model.nv)])

wandb.config.n_col = 5
wandb.config.u_max = [2.5, 2.5, 2.5, 1.5, 1.5, 1.5, 1.0]
wandb.config.dt = 0.05

wandb.config.isvec = True
wandb.config.n_vars = 3*model.nq*wandb.config.n_col + 2*model.nq

wandb.config.lr_nn = 1e-3
wandb.config.batch_size = 64
wandb.config.epochs = 5000

wandb: Currently logged in as: hjzhu. Use `wandb login --relogin` to force relogin


In [11]:
train_data = run.use_artifact('hjzhu/vocam/qpnet_datasets:v1', type='dataset')
train_dir = train_data.download()

In [12]:
train_dir

'./artifacts/qpnet_datasets:v1'

In [12]:
du = DataUtils(robot, wandb.config, viz)
du.load("../data/data_train.pt")

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = QPNet(du.x_train.shape[1], du.y_train.shape[1]).to(device)
optimizer = torch.optim.Adam(network.parameters(), lr=wandb.config.lr_nn)
criterion = torch.nn.L1Loss()
train_dataloader = DataLoader(data_train, batch_size=wandb.config.batch_size, 
                              shuffle=True)
test_dataloader = DataLoader(data_test, batch_size=wandb.config.batch_size, 
                              shuffle=True)
wandb.watch(network)

[]

In [15]:
for epoch in trange(wandb.config.epochs):
    loss_train = vocam.qpnet.train(network, criterion, optimizer, train_dataloader, device)
    loss_test = vocam.qpnet.test(network, criterion, test_dataloader, device)
    
    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train Loss": loss_train,
        "Test Loss": loss_test})
network.save("../models/test1")

100%|██████████| 100/100 [00:31<00:00,  3.17it/s]


In [ ]:
# save trained model as artifact
trained_model_artifact = wandb.Artifact(
            MODEL_NAME, type="model",
            description="trained inception v3",
            metadata=dict(cfg))

model.save(SAVE_MODEL_DIR) # save using Keras
trained_model_artifact.add_dir(SAVE_MODEL_DIR)
run.log_artifact(trained_model_artifact)